In [50]:
import numpy as np 
import pandas as pd
import itertools
from joblib import Parallel, delayed
import argparse
from tqdm import tqdm
from QAOA_CSP import CSP, _H, getBaseProb
from QAOA import _Rx,_Rz,_Rzz,_Rzzz,_Rzzzz,I,_H,getBaseProb
import matplotlib.pyplot as plt

def I(n): #返回可占用n个比特位的I矩阵
    return np.eye(2**n)
z = np.array([[1,0],
                [0,-1]],dtype=np.complex128)
def create_new_dict(nested_dict, func) -> dict:
    new_dict = {}
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            new_dict[key] = create_new_dict(value, func)
        else:
            new_key, new_value = func(key, value)
            new_dict[new_key] = new_value
    
    return new_dict

# 将二层的key转化为我要的形式的字符串
def modify_key_value(key, value):
    operLst = ",".join(list(map(lambda x:str(eval(x[2]) - 1),key.split())))

    return operLst, value
def Qaoa3E(Problem:dict,HC,gammaLst:list,betaLst:list,p:int,s:np.ndarray) -> float:
    model = CSP(Problem,HC)
    model.updateQC(gammaLst,betaLst,p)
    return model.getExpctation(s)

# Problem转为经典表达式
def paraCSP(Problem,z1,z2,z3,z4):
    C = 0
    Problem_beta = Problem[1]
    Constant = Problem[2]
    factor = Problem[3]
    for _, Power in Problem_beta.items(): #每个次幂的
        for key, coef in Power.items(): #某个次幂中 每个项
            # 解析每个项的表达式，并乘value系数
            item = 1
            for oper in key.split(): 
                item  = item * eval("z{}".format(oper[2]))
            C  = C + item*coef 
    return factor*(C+Constant)
def GridSearch_CSP(Problem):
    sulotionsSet = itertools.product([-1,1],repeat=4)
    obj_maxLst = [0]
    para_max = []
    for z1,z2,z3,z4 in sulotionsSet:
        obj = paraCSP(Problem,z1,z2,z3,z4)
        if obj > obj_maxLst[0]:
            obj_maxLst = [obj]
            para_max = [(z1,z2,z3,z4)]
        elif obj == obj_maxLst[0]:
            obj_maxLst.append(obj)
            para_max.append((z1,z2,z3,z4))
        else:
            pass
    return obj_maxLst,para_max


Constant = 0
factor = 1
t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
def getParaQC(Problem,gamma:float, beta:float):
    Problem_alpha = Problem[0]
    Problem_beta = Problem[1]
    QC = 1
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_1"].items(),Problem_beta["Power_1"].items()):
        if coef != 0:
            QC = _Rz(eval(qubit_str),4,coef*gamma) * QC
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_2"].items(),Problem_beta["Power_2"].items()):
        if coef != 0:
            qubit_idx = eval(qubit_str)
            QC = _Rzz(qubit_idx[0],qubit_idx[1],4,coef*gamma) * QC
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_3"].items(),Problem_beta["Power_3"].items()):
        if coef != 0:
            qubit_idx = eval(qubit_str)
            QC = _Rzzz(qubit_idx[0],qubit_idx[1],qubit_idx[2],4,coef*gamma) * QC

    if Problem_alpha["Power_4"]['0,1,2,3'] != 0:
        QC = _Rzzzz(Problem_alpha["Power_4"]['0,1,2,3']*gamma) *  QC
    QC = _Rx(0,4,beta)*_Rx(1,4,beta)*_Rx(2,4,beta)*_Rx(3,4,beta) * QC
    return QC 
def QC_func(Problem, reversed = False): #输出一个func，对应Problem
    def func(X):
        #print(X)
        if X.shape[0] == 4:
            gamma_1,beta_1,gamma_2,beta_2 = X[0],X[1],X[2],X[3]
        elif X.shape[0] ==2:
            gamma_1,beta_1,gamma_2,beta_2 = X[0],X[1],0,0
        else:
            raise ValueError("X 输入不对")
        QC = getParaQC(Problem,gamma_2,beta_2) * getParaQC(Problem,gamma_1,beta_1)
        global s,HC
        rb = QC.dot(s)
        E = (np.conjugate(rb).T.dot(HC).dot(rb))[0][0].real
        if reversed:
            #print(-E)
            return -E
        else:
            return E
    return func

def estimate_gradient(func, X:np.ndarray, epsilon = 1e-8) ->np.ndarray: #Qaoa3E专用
    x_epsilon = np.array([epsilon,0])
    y_epsilon = np.array([0,epsilon])
    E_det_G1 = func(X+x_epsilon)
    E_det_G2 = func(X-x_epsilon)
    gamma_gradient = (E_det_G1 - E_det_G2)/(2*epsilon)
    E_det_B1 = func(X+y_epsilon)
    E_det_B2 = func(X-x_epsilon)
    beta_gradient = (E_det_B1 - E_det_B2)/(2*epsilon)
    return np.array([gamma_gradient,beta_gradient])


In [149]:
CtrlSeries = [1,1,0,-1,0,0,0,0,0,0,-1,0,0,0,1] #代表问题8
Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
        "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
Problem_beta = {"Power_1":Power_1,
        "Power_2":Power_2,
        "Power_3":Power_3,
        "Power_4":Power_4}
Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
Problem = (Problem_alpha,Problem_beta, Constant, factor)
func = QC_func(Problem,True)
model = CSP(Problem)
HC = model.HC


In [150]:
from scipy.optimize import dual_annealing

In [151]:
func = QC_func(Problem,True)
bounds = [[0,2*np.pi],
          [0,np.pi],
          [0,2*np.pi],
          [0,np.pi]]
lt_E = []
X = np.array([4,2,1,2])
def callback(x, f, context):
    
    #plt.scatter(x[0], -f, color='red', marker='o')  # 绘制当前最优解
    lt_E.append(-f)
    
result = dual_annealing(func,bounds,callback=callback,restart_temp_ratio=1e-3,visit = 1.5,x0 = X,maxiter = 1000)

In [153]:
result.fun

-3.1715105830978856

In [156]:
X = result.x

In [70]:
func(np.array([0.76,0.6]))

1.7024312189957966